In [1]:
# !pip install --upgrade mxnet==1.6.0
# !pip install gluonts

In [2]:
# Import libraries
import os
import numpy as np
import torch
import pandas as pd

In [3]:
# Download Electricity Dataset
file = "LD2011_2014_clean.txt"
isExist = os.path.exists(file)
if not isExist:
    !wget https://archive.ics.uci.edu/ml/machine-learning-databases/00321/LD2011_2014.txt.zip
    !unzip LD2011_2014.txt.zip
    # change commas to dots
    !sed 's/,/./g' LD2011_2014.txt > LD2011_2014_clean.txt
else: 
    print(f"Files are ready")

Files are ready


In [4]:
"""Preprocess data"""
data = pd.read_csv('LD2011_2014_clean.txt', delimiter = ';')
#remove data before 2012
data = data.iloc[8760*4:]
print('Data loaded..')
data_2 = data.copy()
#pick the first 20 houses
data_2 = data_2.iloc[:,:]
# Aggregate
data_2['time'] =pd.to_datetime(data_2['Unnamed: 0']).dt.ceil('1h') 
data_2 = data_2.drop(['Unnamed: 0'], axis = 1)
agg_dict = {}
for col in data_2.columns[:-1]:
    agg_dict[col] ='mean'
aggregated_data = data_2.groupby(['time']).agg(agg_dict)
print('Data aggregated by hour')
aggregated_data.shape

Data loaded..
Data aggregated by hour


(26304, 370)

In [5]:
"""Normalize data"""
from sklearn.preprocessing import StandardScaler, MinMaxScaler
Std_normalization = 1
if Std_normalization:
    scaler = StandardScaler()
    temp = scaler.fit_transform(aggregated_data)
    norm_means = scaler.mean_
    norm_std = scaler.scale_
else:
    scaler = MinMaxScaler()
    temp = scaler.fit_transform(aggregated_data)
global data_4
normalized_data = pd.DataFrame(temp, index=aggregated_data.index, columns = aggregated_data.columns)

# Check normalized data
normalized_data.head()

,MT_001,MT_002,MT_003,MT_004,MT_005,MT_006,MT_007,MT_008,MT_009,MT_010,...,MT_361,MT_362,MT_363,MT_364,MT_365,MT_366,MT_367,MT_368,MT_369,MT_370
time,,,,,,,,,,,,,,,,,,,,,
2012-01-01 01:00:00,-0.193637,-0.767048,5.979015,0.750556,1.297119,2.562526,0.380006,0.420689,0.936652,1.237010,...,-0.925467,-0.660711,-0.611351,-0.924599,-1.182866,0.055241,-0.298429,-0.970811,-0.867907,-1.312616
2012-01-01 02:00:00,-0.088516,-0.711666,5.979015,0.723940,1.174705,2.419487,0.131533,0.362440,0.666522,1.016322,...,-0.968145,-0.718147,-0.862776,-0.926923,-1.177477,-0.195217,-0.111780,-0.964607,-0.923605,-1.312616
2012-01-01 03:00:00,-0.193637,-0.739357,5.979015,0.710631,0.964851,1.573172,0.090121,-0.263732,0.500288,0.616982,...,-1.023433,-0.710625,-0.879452,-0.931572,-1.177477,-0.289139,-0.478750,-1.119708,-1.044284,-1.312616
2012-01-01 04:00:00,-0.088516,-0.877811,5.979015,-0.194342,0.037999,0.059340,-0.179057,-0.948152,-0.559453,-0.391876,...,-1.228097,-0.732505,-0.896128,-0.957141,-1.139759,-0.289139,-1.696712,-1.175544,-1.102634,-1.312616
2012-01-01 05:00:00,-0.141076,-0.905502,5.979015,-0.726679,-0.259294,-0.524736,-0.365412,-1.297643,-0.351661,-0.539002,...,-1.179598,-0.753702,-0.901901,-0.961790,-1.182866,-0.383061,-1.707784,-1.169340,-1.072133,-1.312616


In [6]:
from gluonts.mx import DeepAREstimator
from gluonts.mx import Trainer

In [7]:
# Hyper-parameters
n_clients = 370
prediction_length = 24
freq = "1H"
train_start = pd.Period("01-01-2012", freq=freq) 
test_start = pd.Period("18-12-2014", freq=freq)
missing_percentage = 60

/home/tung6100/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3135: UserWarning: Parsing '18-12-2014' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
"""

Create missing values based on normal distribution random choice

Input: 
 - pd_data: 2d pandas data frame
 - missing_percentage: missing percentage <= 100

Output:
 - return_data: 2d pandas with missed values

"""
def create_missing_data(pd_data, missing_percentage = 20):
    if missing_percentage == 0: return pd_data
    assert missing_percentage <= 100, "missing percentage should be less than or equal 100%"
    np.random.seed(1993)
    # Convert data frame to array
    np_data = pd_data.to_numpy()
    # Convert original data to 1d array - Because existing package only supports to randomly choose indices from 1d array
    np_data_1d = np_data.flatten()
    # Randomly choose missing index
    total_elem = np_data_1d.shape[0]
    number_of_missing_elem = int(missing_percentage*1.0*total_elem/100)
    missing_index = np.random.choice(np.arange(total_elem), number_of_missing_elem, replace=False) # with replace = False, an index only is chosen 1 time
    # Replace missing_index with 0
    np_data_1d[missing_index] = 0
    # Convert 1d array to 2d array
    np_data_2d = np_data_1d.reshape(pd_data.shape[0], pd_data.shape[1])
    # Convert 2d array to dataframe
    cols_name = pd_data.columns
    return_data = pd.DataFrame(np_data_2d, columns = cols_name)
    return_data.index = pd_data.index
    return return_data

missing_df = create_missing_data(normalized_data, missing_percentage=20)
missing_df.head()

,MT_001,MT_002,MT_003,MT_004,MT_005,MT_006,MT_007,MT_008,MT_009,MT_010,...,MT_361,MT_362,MT_363,MT_364,MT_365,MT_366,MT_367,MT_368,MT_369,MT_370
time,,,,,,,,,,,,,,,,,,,,,
2012-01-01 01:00:00,-0.193637,0.000000,0.000000,0.750556,1.297119,2.562526,0.380006,0.420689,0.936652,1.237010,...,0.000000,-0.660711,-0.611351,-0.924599,-1.182866,0.000000,-0.298429,0.000000,-0.867907,-1.312616
2012-01-01 02:00:00,-0.088516,-0.711666,5.979015,0.723940,0.000000,0.000000,0.131533,0.362440,0.000000,1.016322,...,0.000000,-0.718147,-0.862776,-0.926923,-1.177477,-0.195217,-0.111780,-0.964607,-0.923605,-1.312616
2012-01-01 03:00:00,-0.193637,-0.739357,0.000000,0.710631,0.000000,0.000000,0.090121,-0.263732,0.500288,0.616982,...,-1.023433,-0.710625,-0.879452,-0.931572,-1.177477,0.000000,0.000000,-1.119708,-1.044284,0.000000
2012-01-01 04:00:00,0.000000,-0.877811,5.979015,-0.194342,0.037999,0.059340,-0.179057,-0.948152,0.000000,-0.391876,...,-1.228097,-0.732505,-0.896128,-0.957141,0.000000,-0.289139,0.000000,-1.175544,0.000000,-1.312616
2012-01-01 05:00:00,0.000000,-0.905502,5.979015,-0.726679,-0.259294,0.000000,-0.365412,-1.297643,-0.351661,-0.539002,...,-1.179598,-0.753702,-0.901901,0.000000,-1.182866,0.000000,-1.707784,-1.169340,0.000000,-1.312616


In [9]:
"""Function to split train and test dataset"""
def set_train_test(n_clients=20):
    data_train = normalized_data.iloc[:25968,:n_clients] #25600; 25968; 26082
    data_test = normalized_data.iloc[25968:25968+24,:n_clients]
    data_test_deepAR = normalized_data.iloc[:25968+prediction_length,:n_clients]
    return data_train, data_test, data_test_deepAR

In [10]:
"""Check train and test dataset"""
data_train, data_test, data_test_deepAR = set_train_test(n_clients=n_clients)
data_train.shape, data_test.shape, data_test_deepAR.shape

((25968, 370), (24, 370), (25992, 370))

In [11]:
# missing_percentage = 40
data_train_missing_vals = create_missing_data(data_train, missing_percentage=missing_percentage)
data_train_missing_vals.head()

,MT_001,MT_002,MT_003,MT_004,MT_005,MT_006,MT_007,MT_008,MT_009,MT_010,...,MT_361,MT_362,MT_363,MT_364,MT_365,MT_366,MT_367,MT_368,MT_369,MT_370
time,,,,,,,,,,,,,,,,,,,,,
2012-01-01 01:00:00,0.0,0.000000,0.000000,0.750556,0.000000,0.0,0.000000,0.000000,0.936652,0.000000,...,0.000000,0.000000,0.000000,-0.924599,-1.182866,0.000000,-0.298429,0.000000,-0.867907,-1.312616
2012-01-01 02:00:00,0.0,-0.711666,5.979015,0.723940,0.000000,0.0,0.000000,0.362440,0.000000,0.000000,...,0.000000,-0.718147,-0.862776,-0.926923,-1.177477,0.000000,-0.111780,-0.964607,-0.923605,-1.312616
2012-01-01 03:00:00,0.0,-0.739357,0.000000,0.000000,0.000000,0.0,0.000000,-0.263732,0.500288,0.616982,...,0.000000,0.000000,-0.879452,0.000000,0.000000,0.000000,0.000000,0.000000,-1.044284,0.000000
2012-01-01 04:00:00,0.0,0.000000,0.000000,0.000000,0.037999,0.0,-0.179057,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-0.957141,0.000000,-0.289139,0.000000,-1.175544,0.000000,0.000000
2012-01-01 05:00:00,0.0,-0.905502,0.000000,0.000000,-0.259294,0.0,0.000000,0.000000,0.000000,-0.539002,...,-1.179598,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.169340,0.000000,0.000000


In [12]:
data_train_missing_vals.iloc[:, :n_clients].to_numpy().shape

(25968, 370)

In [13]:
"""Convert data to numpy"""
data_train_np = data_train_missing_vals.iloc[:, :n_clients].to_numpy()
data_test_np = data_test.iloc[:, :n_clients].to_numpy()
data_test_deepAR_np = data_test_deepAR.iloc[:, :n_clients].to_numpy()
data_train_np = data_train_np.T
data_test_np = data_test_np.T
data_test_deepAR_np = data_test_deepAR_np.T
# Check the type of data
type(data_train_np), data_train_np.shape, data_test_np.shape, data_test_deepAR_np.shape

(numpy.ndarray, (370, 25968), (370, 24), (370, 25992))

In [14]:
"""Create dataset for Gluont"""
from gluonts.dataset.common import ListDataset
# train dataset: cut the last window of length "prediction_length", add "target" and "start" fields
train_ds = ListDataset(
    [{"target": x, "start": train_start} for x in data_train_np[:, :-prediction_length]],
    freq=freq,
)
# test dataset: use the whole dataset, add "target" and "start" fields
test_ds = ListDataset(
    [{"target": x, "start": train_start} for x in data_test_deepAR_np], freq=freq
)

In [15]:
estimator = DeepAREstimator(freq=freq,
                            prediction_length=prediction_length,
                            trainer=Trainer(ctx="cpu", epochs=5, learning_rate=1e-3, num_batches_per_epoch=100),
                            )

In [16]:
predictor = estimator.train(training_data=train_ds)

100%|██████████| 100/100 [00:26<00:00,  3.83it/s, epoch=5/5, avg_epoch_loss=0.49]


In [17]:
from gluonts.evaluation import make_evaluation_predictions
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_ds,  
    predictor=predictor,  
    num_samples=1, 
)

In [18]:
from tqdm.autonotebook import tqdm

print("Obtaining time series conditioning values ...")
tss = list(tqdm(ts_it, total=data_test_deepAR_np.shape[1]))
print("Obtaining time series predictions ...")
forecasts = list(tqdm(forecast_it, total=data_test_deepAR_np.shape[1]))

Obtaining time series conditioning values ...


  0%|          | 0/25992 [00:00<?, ?it/s]

Obtaining time series predictions ...


  0%|          | 0/25992 [00:00<?, ?it/s]

In [19]:
len(tss)

370

In [20]:
import matplotlib.pyplot as plt

def plot_prob_forecasts(ts_entry, forecast_entry):
    plot_length = prediction_length
    prediction_intervals = (80.0, 95.0)
    legend = ["observations", "median prediction"] + [f"{k}% prediction interval" for k in prediction_intervals][::-1]

    fig, ax = plt.subplots(1, 1, figsize=(10, 7))
    ts_entry[-plot_length:].plot(ax=ax)  
    forecast_entry.plot(prediction_intervals=prediction_intervals, color='g')
    plt.grid(which="both")
    plt.legend(legend, loc="upper left")
    plt.show()

In [21]:
for i in tqdm(range(n_clients)):
    ts_entry = tss[i]
    forecast_entry = forecasts[i]
    # plot_prob_forecasts(ts_entry, forecast_entry)

  0%|          | 0/370 [00:00<?, ?it/s]

In [22]:
list_diff = []
y_pred = []
y_true = []
for i in tqdm(range(n_clients)):
    ts_entry = tss[i]
    ts_entry_np = ts_entry.to_numpy()
    ts_entry_24h = ts_entry_np[-24:, :]
    ts_entry_24h = ts_entry_24h.reshape(-1)
    forecast_entry_24h = forecasts[i].mean
    diff = ts_entry_24h - forecast_entry_24h
    y_pred.append(forecast_entry_24h)
    y_true.append(ts_entry_24h)
    list_diff.append(diff)

y_pred_np = np.array(y_pred)
y_true_np = np.array(y_true)

y_pred_np.shape, y_true_np.shape

  0%|          | 0/370 [00:00<?, ?it/s]

((370, 24), (370, 24))

In [23]:
dif_matrix = np.array(list_diff)
dif_matrix.shape

(370, 24)

In [24]:
rmse = np.sqrt(np.mean(np.square(dif_matrix)))
rmse

2.6396759

In [25]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
def evaluate_Testset(y_true, y_pred):
    def mape(y_true, y_pred): 
      return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    MSE = mse(y_true, y_pred)
    RMSE = np.sqrt(MSE)
    MAE = mae(y_true, y_pred)
    MAPE = mape(y_true, y_pred)
    return MSE, RMSE, MAE, MAPE

def accuracy_Testset(y_true, y_pred):
  diff_pred = list()
  diff_true = list()
  accuracy = 0.
  for i in range(len(y_true)-1):
      diff_pred.append(y_pred[i+1]-y_pred[i])
      diff_true.append(y_true[i+1]-y_true[i])
  count = sum(diff_pred[i] * diff_true[i] > 0 for i in range(len(diff_pred)))
  accuracy = count/len(diff_pred) * 100
  return accuracy

In [26]:
MSE, RMSE, MAE, MAPE = evaluate_Testset(y_true_np, y_pred_np)
acc = accuracy_Testset(y_true_np, y_pred_np)
print(f"MSE: {MSE}, RMSE: {RMSE}, MAE: {MAE}, MAPE: {MAPE}")
# print(f"acc: {acc}")

MSE: 6.967889308929443, RMSE: 2.639676094055176, MAE: 1.159835934638977, MAPE: 270.5078601837158
